In [94]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [95]:
df_industries = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/industries_indexes.csv',encoding = 'iso-8859-1')
df_industries['Industry imba dependency'] = df_industries['Industry trade dependency']
df_industries['Import-Export imbalance'] = df_industries['Import-Export imbalance']*100
df_industries['Import Growth'] = df_industries['Import Growth']*100
df_industries['Export Growth'] = df_industries['Export Growth']*100

In [97]:
regions = df_industries['Area'].unique().tolist()
industries = df_industries['Industry'].unique().tolist()
years = np.unique(df_industries['Time'])
future_years = [2021, 2022, 2023]
time = [2015,2016,2017,2018,2019,2020,2021,2022,2023]

In [98]:
# Exports by industry
all_export_growth_preds = []
all_exports = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        
        if filtered_df.shape[0] == 0:
            continue
        #LOOCV
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            # Features
            X_train = train_data[['Time']]
            y_train = train_data['Export Growth']
            export_growth_model = LinearRegression()
            export_growth_model.fit(X_train, y_train)
            # Predictions for test set
            export_growth_pred = export_growth_model.predict(test_data[['Time']])
            # Mean squared error for LOOCV
            mse = mean_squared_error(test_data['Export Growth'], export_growth_pred)
            # Identify best model with LOOCV
            if mse < best_mse:
                best_model = export_growth_model
                best_mse = mse
        # Use the best model to make predictions for 3 years in the future
        future_preds_export_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            export_growth_pred = best_model.predict(X_future)
            future_preds_export_growth.append(export_growth_pred[0])
        # Saving results
        all_exports.append(filtered_df['Export Growth'].tolist() + future_preds_export_growth)

# Saving results to csv
df_exports = pd.DataFrame()
df_exports['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_exports[column_name] = all_exports[index]
        index += 1
df_exports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/exports_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_864\40375563.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exports[column_name] = all_exports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\40375563.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_exports[column_name] = all_exports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\40375563.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [99]:
# Do the same for imports
all_import_growth_preds = []
all_imports = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import Growth']
            import_growth_model = LinearRegression()
            import_growth_model.fit(X_train, y_train)
            import_growth_pred = import_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import Growth'], import_growth_pred)
            if mse < best_mse:
                best_model = import_growth_model
                best_mse = mse
        future_preds_import_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            import_growth_pred = best_model.predict(X_future)
            future_preds_import_growth.append(import_growth_pred[0])
        all_imports.append(filtered_df['Import Growth'].tolist() + future_preds_import_growth)
df_imports = pd.DataFrame()
df_imports['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imports[column_name] = all_imports[index]
        index += 1
df_imports.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imports_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1813326717.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports[column_name] = all_imports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1813326717.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imports[column_name] = all_imports[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1813326717.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Conside

In [101]:
# Do the same for trade dependencies
all_trade_growth_preds = []
all_trades = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Industry trade dependency']
            trade_growth_model = LinearRegression()
            trade_growth_model.fit(X_train, y_train)
            trade_growth_pred = trade_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Industry trade dependency'], trade_growth_pred)
            if mse < best_mse:
                best_model = trade_growth_model
                best_mse = mse
        future_preds_trade_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            trade_growth_pred = best_model.predict(X_future)
            future_preds_trade_growth.append(trade_growth_pred[0])
        all_trades.append(filtered_df['Industry trade dependency'].tolist() + future_preds_trade_growth)
df_trades = pd.DataFrame()
df_trades['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_trades[column_name] = all_trades[index]
        index += 1
df_trades.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/trade_dependencies_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1888505705.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_trades[column_name] = all_trades[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1888505705.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_trades[column_name] = all_trades[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\1888505705.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

In [102]:
# Do the same for import-export imbalance
all_imba_growth_preds = []
all_imbas = []
for region in regions:
    for industry in industries:
        condition1 = df_industries['Area'] == region
        condition2 = df_industries['Industry'] == industry
        filtered_df = df_industries[condition1 & condition2]
        if filtered_df.shape[0] == 0:
            continue
        loo = LeaveOneOut()
        best_model = None
        best_mse = np.inf  
        for train_index, test_index in loo.split(filtered_df):
            train_data = filtered_df.iloc[train_index]
            test_data = filtered_df.iloc[test_index]
            X_train = train_data[['Time']]
            y_train = train_data['Import-Export imbalance']
            imba_growth_model = LinearRegression()
            imba_growth_model.fit(X_train, y_train)
            imba_growth_pred = imba_growth_model.predict(test_data[['Time']])
            mse = mean_squared_error(test_data['Import-Export imbalance'], imba_growth_pred)
            if mse < best_mse:
                best_model = imba_growth_model
                best_mse = mse
        future_preds_imba_growth = []
        for year in future_years:
            X_future = pd.DataFrame({'Time': [year]})
            imba_growth_pred = best_model.predict(X_future)
            future_preds_imba_growth.append(imba_growth_pred[0])
        all_imbas.append(filtered_df['Import-Export imbalance'].tolist() + future_preds_imba_growth)
df_imbas = pd.DataFrame()
df_imbas['Time'] = time
index = 0
for region in regions:
    for industry in industries:
        column_name = f"{region}_{industry}"
        df_imbas[column_name] = all_imbas[index]
        index += 1
df_imbas.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/forecast_values/imbalances_industry.csv')

C:\Users\hamin\AppData\Local\Temp\ipykernel_864\3264172383.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imbas[column_name] = all_imbas[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\3264172383.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_imbas[column_name] = all_imbas[index]
C:\Users\hamin\AppData\Local\Temp\ipykernel_864\3264172383.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joinin